# Tesla & GameStop Stock and Revenue Analysis

This notebook extracts and visualizes financial data for Tesla and GameStop using `yfinance` and web scraping techniques, then builds dashboards comparing stock prices to revenue.


# Question 1: Extracting Tesla Stock Data Using yfinance


In [2]:
import yfinance as yf

# Download Tesla historical data
tesla = yf.Ticker('TSLA')
tesla_stock = tesla.history(period='max')
tesla_stock.reset_index(inplace=True)
tesla_stock.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


# Question 2: Extracting Tesla Revenue Data Using Webscraping


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}
response = requests.get(url, headers=headers, timeout=30)
response.raise_for_status()
html = response.text
soup = BeautifulSoup(html, "html.parser")
tables = soup.find_all('table')

# Find the table with 'Tesla Quarterly Revenue'
tesla_revenue_table = None
for table in tables:
    if "Tesla Quarterly Revenue" in str(table):
        tesla_revenue_table = table
        break

if tesla_revenue_table is None:
    raise ValueError("Tesla revenue table not found on Macrotrends page")

tesla_revenue = pd.read_html(StringIO(str(tesla_revenue_table)))[0]
tesla_revenue.columns = ['Date', 'Revenue']
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != '-']  # Remove empty rows

tesla_revenue.head()


,Date,Revenue
0,2025-09-30,"$28,095"
1,2025-06-30,"$22,496"
2,2025-03-31,"$19,335"
3,2024-12-31,"$25,707"
4,2024-09-30,"$25,182"


# Question 3: Extracting GameStop Stock Data Using yfinance


In [4]:
# Download GameStop historical data
gamestop = yf.Ticker('GME')
gme_stock = gamestop.history(period='max')
gme_stock.reset_index(inplace=True)
gme_stock.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


# Question 4: Extracting GameStop Revenue Data Using Webscraping


In [5]:
# Scrape GameStop revenue from MacroTrends
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
response = requests.get(url, headers=headers, timeout=30)
response.raise_for_status()
html = response.text
soup = BeautifulSoup(html, "html.parser")
tables = soup.find_all('table')

# Find the table with 'GameStop Quarterly Revenue'
gme_revenue_table = None
for table in tables:
    if "GameStop Quarterly Revenue" in str(table):
        gme_revenue_table = table
        break

if gme_revenue_table is None:
    raise ValueError("GameStop revenue table not found on Macrotrends page")

gme_revenue = pd.read_html(StringIO(str(gme_revenue_table)))[0]
gme_revenue.columns = ['Date', 'Revenue']
gme_revenue = gme_revenue[gme_revenue['Revenue'] != '-']  # Remove empty rows

gme_revenue.head()


,Date,Revenue
0,2025-07-31,$972
1,2025-04-30,$732
2,2025-01-31,"$1,283"
3,2024-10-31,$860
4,2024-07-31,$798


# Question 5: Tesla Stock and Revenue Dashboard


In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Clean and prepare data
tesla_stock['Date'] = pd.to_datetime(tesla_stock['Date'])
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(',', '').str.replace('$', '').astype(float)
tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'])

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add stock price line
fig.add_trace(
    go.Scatter(x=tesla_stock['Date'], y=tesla_stock['Close'], name="Tesla Stock Price", line=dict(color='blue')),
    secondary_y=False,
)

# Add revenue bars
fig.add_trace(
    go.Bar(x=tesla_revenue['Date'], y=tesla_revenue['Revenue'], name="Tesla Revenue", marker=dict(color='green')),
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Stock Price (USD)", secondary_y=False)
fig.update_yaxes(title_text="Revenue (USD)", secondary_y=True)

# Update layout
fig.update_layout(
    title_text="Tesla Stock Price vs Revenue",
    legend=dict(x=0.01, y=0.99)
)

fig.show()


# Question 6: GameStop Stock and Revenue Dashboard


In [7]:
# Clean and prepare GameStop data
gme_stock['Date'] = pd.to_datetime(gme_stock['Date'])
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',', '').str.replace('$', '').astype(float)
gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'])

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add stock price line
fig.add_trace(
    go.Scatter(x=gme_stock['Date'], y=gme_stock['Close'], name="GameStop Stock Price", line=dict(color='red')),
    secondary_y=False,
)

# Add revenue bars
fig.add_trace(
    go.Bar(x=gme_revenue['Date'], y=gme_revenue['Revenue'], name="GameStop Revenue", marker=dict(color='orange')),
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Stock Price (USD)", secondary_y=False)
fig.update_yaxes(title_text="Revenue (USD)", secondary_y=True)

# Update layout
fig.update_layout(
    title_text="GameStop Stock Price vs Revenue",
    legend=dict(x=0.01, y=0.99)
)

fig.show()


# Question 7: Sharing your Assignment Notebook

## Instructions for Submission:

1. **Run all cells** to ensure code executes without errors and produces correct results
2. **Take screenshots** for each question (Q1-Q6) showing:
   - The question header (Markdown cell)
   - The code cell
   - The output/results from executing that code
3. **Save your notebook** with all outputs included
4. **Submit both**:
   - The completed `.ipynb` notebook file
   - Screenshots for each question showing code and results

### Required Packages:
Make sure you have installed the following packages before running:
- `yfinance`
- `pandas`
- `requests`
- `beautifulsoup4`
- `plotly`

Install with: `pip install yfinance pandas requests beautifulsoup4 plotly`
